In [1]:
import sys, os, importlib
import rasterio, folium

import pandas as pd
import geopandas as gpd

from shapely.geometry import Point, LineString

In [8]:
out_folder = '/home/wb411133/data/Projects/BFA_Energy/Data'

# Map villages

In [10]:
# Create point data for all villages
in_data = "All_Data.xlsx"
inD = pd.read_excel(in_data, sheet_name=0)

# Convert X and Y to coordinates
inD['X'] = inD['X'].apply(lambda x: float(x.replace(" ",'').replace(",",".")))
inD['Y'] = inD['Y'].apply(lambda x: float(x.replace(" ",'').replace(",",".")))

inD.groupby("UTM")
all_res = []
for idx, grp in inD.groupby("UTM"):
    geoms = [Point(x) for x in zip(grp['X'],inD['Y'])]
    crs_idx = int(f'326{idx[:2]}')
    grp_D = gpd.GeoDataFrame(grp, geometry=geoms, crs=crs_idx)
    grp_D = grp_D.to_crs(4326)
    all_res.append(grp_D)
    
final_villages = gpd.GeoDataFrame(pd.concat(all_res), geometry='geometry', crs=4326)
final_villages.to_file(os.path.join(out_folder, "all_villages.geojson"), driver="GeoJSON")

In [11]:
final_villages.shape

(179, 7)

In [7]:
map_osm = folium.Map(location=[final_villages.unary_union.centroid.y, final_villages.unary_union.centroid.x])
'''tile = folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = False,
        control = True
       ).add_to(map_osm)
'''
points = folium.features.GeoJson(final_villages.to_json())
map_osm.add_child(points)
map_osm

# Map connection lines

In [2]:
in_data = "All_Data.xlsx"
inD = pd.read_excel(in_data, sheet_name=1)

inD.groupby("UTM")
all_res = []
for idx, grp in inD.groupby("UTM"):
    geoms = [Point(x) for x in zip(grp['X'],inD['Y'])]
    crs_idx = int(f'326{idx}')
    grp_D = gpd.GeoDataFrame(grp, geometry=geoms, crs=crs_idx)
    grp_D = grp_D.to_crs(4326)
    all_res.append(grp_D)
    
line_points = gpd.GeoDataFrame(pd.concat(all_res), geometry='geometry', crs=4326)

In [3]:
all_res = []
for idx, pts in line_points.groupby(["Name",'UTM']):
    print(idx)    
    pts = pts.reset_index()
    pts = pts.sort_values(['Order'])
    all_res.append([idx[0],idx[1],LineString(pts['geometry'])])
    if idx == "4B-TL":
        break
    
line_lines = gpd.GeoDataFrame(pd.DataFrame(all_res, columns=['Name','UTM','geometry']), geometry='geometry', crs=4326)

('1A-TL', 31)
('1B-TL', 31)
('1C-TL', 31)
('2-TL', 31)
('3-TL', 31)
('4A-TL', 31)
('4B-TL', 30)
('4B-TL', 31)
('5-TL-OE-OSE', 30)
('5-TL-OSE-PA5', 30)
('5-TL-PA5', 30)


In [5]:
line_lines.to_file(os.path.join("/home/wb411133/data/Projects/BFA_Energy/Data/transmission_lines.shp"))

In [ ]:
map_osm = folium.Map(location=[line_points.unary_union.centroid.y, line_points.unary_union.centroid.x])
'''tile = folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = False,
        control = True
       ).add_to(map_osm)
'''
points = folium.features.GeoJson(line_points.to_json(), tooltip=folium.features.GeoJsonTooltip(fields=['Points']))
#map_osm.add_child(points)
lines = folium.features.GeoJson(line_lines.to_json(), tooltip=folium.features.GeoJsonTooltip(fields=['Name']))
map_osm.add_child(lines)
map_osm

In [ ]:
line_points.head()

# Map pilot cities

In [ ]:
in_data = [['Gaboanli','Village'  ,240031,1362753,'31N'],
           ['Sakoani','Village'   ,317254.55,1371825.72,'31N'],
           ['Tialboanga','Village',312678.00,1365840.00,'31N']]

inD = pd.DataFrame(in_data, columns=['Location_Name','Type','UTM_E','UTM_N','UTM_Zone'])

In [ ]:
geoms = [Point(x) for x in zip(inD['UTM_E'],inD['UTM_N'])]
inD = gpd.GeoDataFrame(inD, geometry=geoms, crs=32631)
inD = inD.to_crs(4326)
inD

In [ ]:
map_osm = folium.Map(location=[inD.unary_union.centroid.y, inD.unary_union.centroid.x])
tile = folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = False,
        control = True
       ).add_to(map_osm)
polys = folium.features.GeoJson(inD.buffer(0.005).to_json())
map_osm.add_child(polys)
points = folium.features.GeoJson(inD.to_json())
map_osm.add_child(points)
map_osm

In [ ]:
inD.to_csv("Pilot_villages.csv")

## Extract buildings for each village

In [ ]:
buildings_file = '/home/public/Data/COUNTRY/BFA/DigitizeAfrica/AFRICA_BURKINA_FASO_building_32631_year2.shp'
inB = gpd.read_file(buildings_file)
sindex = inB.sindex

In [ ]:
sindex = inB.sindex

In [ ]:
all_res = []
inD = inD.to_crs(inB.crs)
for idx, row in inD.iterrows():
    row['geometry'] = row['geometry'].buffer(1000)
    potential_buildings = inB.loc[list(sindex.intersection(row['geometry'].bounds))]
    all_res.append(potential_buildings)
sel_buildings = pd.concat(all_res)

In [ ]:
sel_buildings = sel_buildings.to_crs(4326)

In [ ]:
map_osm = folium.Map(location=[sel_buildings.unary_union.centroid.y, sel_buildings.unary_union.centroid.x])
'''
tile = folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = False,
        control = True
       ).add_to(map_osm)
'''
points = folium.features.GeoJson(sel_buildings.to_json())
map_osm.add_child(points)
map_osm

In [ ]:
sel_buildings.to_file("/home/wb411133/temp/BFA_buildings.shp")